In [3]:
estimates_vs_actuals_task_to_story = """
select	
	task.Name as TaskName,
	project.Name as ProjectName,
	story.Name as StoryName,
	user.DisplayName,
	user.EmailAddress,
	task.Actuals as TaskActuals_hrs,
	task.Estimate as TaskEstimate_hrs,
	story.PlanEstimate as StoryEstimate_pts
from
	task
left join
	story
on
	task.StoryId = story.Id
left join
	project
on
	task.ProjectId = project.Id
left join
	user
on
	task.OwnerId = user.Id
where
	task.State = 'Completed'
"""

estimates_vs_actuals_story_to_function = """
select	
	function.Name as FunctionName,
	project.Name AS ProjectName,
	story.Name AS StoryName,
	CAST(function.PreliminaryEstimate as int) as FunctionPrelimEstimate_pts,
	SUM(task.Actuals) as StoryActuals_hrs,
    SUM(task.Estimate) as StoryEstimateByTask_hrs,
    story.PlanEstimate as StoryEstimate_pts
from
	task
left join
	story
on
	task.StoryId = story.Id
left join
	project
on
	task.ProjectId = project.Id
left join
	function
on
	story.FunctionId = function.Id
where
	task.State = 'Completed'
and
	function.Name is not null
and
    function.ActualEndDate is not null
and
    story.PlanEstimate > 0
group by
	function.Name,
	project.Name,
	function.PreliminaryEstimate,
	story.Name
having
    sum(task.Actuals) > 0 and sum(task.Estimate) > 0
"""

insprint_defects = """
select
	project.Name  as ProjectName,
	iteration.Name  as IterationName,
	DATETIME(iteration.StartDate) AS IterationStartDate,
	DATETIME(iteration.EndDate) as IterationEndDate,
	defect.Name  as DefectName,
	case when 
	defect.Severity = 'Major Problem' then 2 
	else case when defect.severity = 'Minor Problem' then 1
	else case when defect.Name is not null then 0
	end end end as DefectSeverity
from
	iteration
left join
	defect
on
	defect.IterationId = iteration.Id
inner join
	project
on
	iteration.ProjectId = project.Id
where
    iteration.EndDate < date('now')
"""

In [4]:
import sqlalchemy
import pandas as pd


def connect_to_data():
    engine = sqlalchemy.create_engine(
        'sqlite:///' + r'C:\Users\Tyler Hughes\Documents\Kingsmen Repositories\bannerman\bannerman\bannerman.db'
    )
    return engine.connect()

db = connect_to_data()

task_to_story_df = pd.read_sql_query(
        sql=estimates_vs_actuals_task_to_story,
        con=db
    )

story_to_function_df = pd.read_sql_query(
        sql=estimates_vs_actuals_story_to_function,
        con=db
    )

defect_df = pd.read_sql_query(
        sql=insprint_defects,
        con=db
    )

In [5]:
tts_df = task_to_story_df.copy().groupby(
    ['ProjectName', 'StoryName']
).sum().reset_index().drop(['StoryEstimate_pts'], axis=1)

tts_user_df = task_to_story_df.copy().groupby(
    ['ProjectName', 'DisplayName', 'StoryName', 'EmailAddress']
).sum().reset_index().drop(['StoryEstimate_pts'], axis=1)

join_table = task_to_story_df[task_to_story_df['StoryEstimate_pts'] > 0].copy().drop(
        ['TaskName', 'DisplayName', 'EmailAddress', 'TaskActuals_hrs', 'TaskEstimate_hrs'],
        axis=1
    ).drop_duplicates().reset_index()

tts_df = tts_df.merge(join_table, on=['ProjectName', 'StoryName'])

tts_df['ActualVsEstimateDiff'] = tts_df['TaskActuals_hrs'] - tts_df['TaskEstimate_hrs']
tts_df['EstimatedHoursPerPoint'] = tts_df['TaskEstimate_hrs']/tts_df['StoryEstimate_pts']
tts_df['ActualHoursPerPoint'] = tts_df['TaskActuals_hrs']/tts_df['StoryEstimate_pts']

In [6]:
tts_report_full = tts_df.copy()

tts_report_full.dropna(inplace=True)

tts_report_summary = tts_report_full.copy()[
    (tts_report_full['EstimatedHoursPerPoint'] > 0) & (tts_report_full['ActualHoursPerPoint'] > 0)
].drop(
    ['TaskActuals_hrs','TaskEstimate_hrs', 'StoryEstimate_pts'],
    axis=1
).groupby(['ProjectName']).mean().reset_index().drop('index', axis=1)

tts_report_summary

,ProjectName,ActualVsEstimateDiff,EstimatedHoursPerPoint,ActualHoursPerPoint
0,Compliance Workbench,-1.166364,4.838909,4.415909
1,Market Data Connection,-2.213956,5.665125,4.787891
2,Seurat,-1.022293,6.057962,5.376327
3,TSquare,0.068000,5.399067,5.605633


In [7]:
stf_df = story_to_function_df.copy().groupby(['ProjectName', 'FunctionName']).sum().reset_index().drop(
    ['FunctionPrelimEstimate_pts'], axis=1)

join_table = story_to_function_df[story_to_function_df['FunctionPrelimEstimate_pts'] > 0].copy()
join_table = join_table.drop(
        ['StoryName', 'StoryActuals_hrs', 'StoryEstimateByTask_hrs', 'StoryEstimate_pts'],
        axis=1
    ).drop_duplicates().reset_index().drop('index', axis=1)

stf_df = stf_df.merge(join_table, on=['ProjectName', 'FunctionName'])

stf_df['StoryFunctionEstimateDiff'] = stf_df['StoryEstimate_pts'] - stf_df['FunctionPrelimEstimate_pts']
stf_df['ActualVsEstimateTasksTotal'] = stf_df['StoryActuals_hrs'] - stf_df['StoryEstimateByTask_hrs']
stf_df['TaskActualHoursPerStoryEstimate'] = stf_df['StoryActuals_hrs']/stf_df['StoryEstimate_pts']
stf_df['TaskEstimatedHoursPerStoryEstimate'] = stf_df['StoryEstimateByTask_hrs']/stf_df['StoryEstimate_pts']
stf_df['TaskActualHoursPerFunctionPrelimEstimate'] = stf_df['StoryActuals_hrs']/stf_df['FunctionPrelimEstimate_pts']
stf_df['TaskEstimatedHoursPerFunctionPrelimEstimate'] = stf_df['StoryEstimateByTask_hrs']/stf_df['FunctionPrelimEstimate_pts']

stf_report = stf_df.copy().dropna().drop(['StoryActuals_hrs', 'StoryEstimateByTask_hrs', 'StoryEstimate_pts',
                                         'FunctionPrelimEstimate_pts'], axis=1).groupby(['ProjectName']).mean().reset_index()
stf_df

,ProjectName,FunctionName,StoryActuals_hrs,StoryEstimateByTask_hrs,StoryEstimate_pts,FunctionPrelimEstimate_pts,StoryFunctionEstimateDiff,ActualVsEstimateTasksTotal,TaskActualHoursPerStoryEstimate,TaskEstimatedHoursPerStoryEstimate,TaskActualHoursPerFunctionPrelimEstimate,TaskEstimatedHoursPerFunctionPrelimEstimate
0,Compliance Workbench,Application Appearance,15.00,16.0,3.0,5.0,-2.0,-1.00,5.000000,5.333333,3.000000,3.200000
1,Compliance Workbench,Application Infrastructure,18.00,25.0,5.0,10.0,-5.0,-7.00,3.600000,5.000000,1.800000,2.500000
2,Compliance Workbench,Configuration Item Admin,12.00,20.0,4.0,5.0,-1.0,-8.00,3.000000,5.000000,2.400000,4.000000
3,Compliance Workbench,Control Point Admin,12.00,20.0,4.0,5.0,-1.0,-8.00,3.000000,5.000000,2.400000,4.000000
4,Compliance Workbench,Control Room Dashboard,6.00,6.0,1.0,10.0,-9.0,0.00,6.000000,6.000000,0.600000,0.600000
5,Compliance Workbench,Control Room User Experience - R4,123.00,139.0,28.0,30.0,-2.0,-16.00,4.392857,4.964286,4.100000,4.633333
6,Compliance Workbench,Dashboard Admin,19.00,20.0,4.0,5.0,-1.0,-1.00,4.750000,5.000000,3.800000,4.000000
7,Compliance Workbench,Email Service,46.00,46.0,10.0,10.0,0.0,0.00,4.600000,4.600000,4.600000,4.600000
8,Compliance Workbench,Email Templates,28.00,26.0,12.0,10.0,2.0,2.00,2.333333,2.166667,2.800000,2.600000
9,Compliance Workbench,Event Service,16.00,23.0,5.0,5.0,0.0,-7.00,3.200000,4.600000,3.200000,4.600000


In [11]:
def_df = defect_df[['ProjectName', 'IterationName', 'IterationStartDate']].sort_values(
    ['ProjectName','IterationStartDate'], ascending=False).reset_index()
rolling_count = def_df.groupby(['ProjectName', 'IterationName']).cumcount()
rolling_count.name = 'RollCount'
def_df = def_df.join(rolling_count)
last_5_df = def_df[def_df['RollCount'] < 5].drop(['index', 'IterationStartDate', 'RollCount'], axis=1)

defect_df = defect_df.merge(last_5_df, how='inner', on=['ProjectName', 'IterationName']).drop_duplicates()
if_def_report = defect_df[defect_df.DefectName != None].groupby(['ProjectName', 'DefectSeverity']).count().drop(
    ['IterationName', 'IterationStartDate', 'IterationEndDate'], axis=1
    )

def_df

,index,ProjectName,IterationName,IterationStartDate,RollCount
0,43,TSquare,Iteration 35,2018-03-05 05:00:00,0
1,44,TSquare,Iteration 34,2018-02-19 05:00:00,0
2,46,TSquare,Iteration 34,2018-02-19 05:00:00,1
3,48,TSquare,Iteration 17,2017-05-29 04:00:00,0
4,50,TSquare,Iteration 17,2017-05-29 04:00:00,1
5,28,Seurat,Iteration 2018.4 (2.2.4),2018-03-21 04:00:00,0
6,29,Seurat,Iteration 2018.3 (2.2.3),2018-03-07 05:00:00,0
7,32,Seurat,Iteration 2018.3 (2.2.3),2018-03-07 05:00:00,1
8,35,Seurat,Iteration 2018.3 (2.2.3),2018-03-07 05:00:00,2
9,38,Seurat,Iteration 2018.2 (2.2.2),2018-02-21 05:00:00,0


In [222]:
tts_final1 = pd.melt(
    tts_report_summary,
    id_vars = ['ProjectName'],
    value_vars = list(tts_report.columns).remove('ProjectName')
).set_index(['ProjectName', 'variable']).sort_index()

tts_final1.index.names = ['Project', 'Metric']

tts_final2 = pd.melt(
    tts_report_summary,
    id_vars = ['ProjectName'],
    value_vars = list(tts_report.columns).remove('ProjectName')
).set_index(['variable', 'ProjectName']).sort_index()

tts_final2.index.names = ['Metric', 'Project']

In [231]:
stf_final1 = pd.melt(
    stf_report,
    id_vars = ['ProjectName'],
    value_vars = list(stf_report.columns).remove('ProjectName')
).set_index(['ProjectName','variable']).sort_index()

stf_final1.index.names = ['Project', 'Metric']

stf_final2 = pd.melt(
    stf_report,
    id_vars = ['ProjectName'],
    value_vars = list(stf_report.columns).remove('ProjectName')
).set_index(['variable', 'ProjectName']).sort_index()

stf_final2.index.names = ['Metric', 'Project']

In [232]:
stf_final2

value
Metric                                      Project                         
ActualVsEstimateTasksTotal                  Compliance Workbench   -5.951389
                                            Market Data Connection -9.647030
                                            Seurat                 -5.967391
                                            TSquare                -0.611111
StoryFunctionEstimateDiff                   Compliance Workbench    0.347222
                                            Market Data Connection -2.737624
                                            Seurat                 -5.369565
                                            TSquare                 0.055556
TaskActualHoursPerFunctionPrelimEstimate    Compliance Workbench    4.067824
                                            Market Data Connection  4.859133
                                            Seurat                  4.484964
                                            TSquare                 2.055556
TaskActualHoursPerStoryEstimate             Compliance Workbench    4.276956
                                            Market Data Connection  4.927062
                                            Seurat                  5.532771
                                            TSquare                 3.590741
TaskEstimatedHoursPerFunctionPrelimEstimate Compliance Workbench    4.625463
                                            Market Data Connection  5.776271
                                            Seurat                  4.985507
                                            TSquare                 2.038889
TaskEstimatedHoursPerStoryEstimate          Compliance Workbench    4.849363
                                            Market Data Connection  5.638593
                                            Seurat                  6.334758
                                            TSquare                 3.529630

In [226]:
if_def_report

DefectName
ProjectName            DefectSeverity            
Compliance Workbench   0                        3
Market Data Connection 1                        3
Seurat                 0                        4
TSquare                1                        1
                       2                        2